# I Started this work after reviewing this article on Analytics Vidya. I ran that code on my MacBook Pro after downleading the data.  It used an AlexNet model and everything worked well.  Here is the link to that work by Yamini Ane

https://www.analyticsvidhya.com/blog/2023/02/plant-disease-classification-using-alexnet/ 

# I decided I'd try VGG16 with this data and also looking at the 'test' data in a little more detail 

In [7]:
import numpy as np
import pandas as pd 
#import matplotlib.pyplot as plt
#import tensorflow as tf
from tensorflow.keras import layers
#from time import perf_counter 
import os
#import seaborn as sns
from PIL import Image 
from PIL import ImageEnhance
from skimage.io import imread
import matplotlib.pyplot as plt
import os, random, pathlib, warnings, itertools, math
warnings.filterwarnings("ignore")
import tensorflow as tf
#import tensorflow.keras.backend as K
#from sklearn.metrics import confusion_matrix
from tensorflow.keras import models
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import load_img, img_to_array

from tensorflow.keras.layers import Dense, Dropout, Flatten, Input, LeakyReLU
from tensorflow.keras.layers import BatchNormalization, Activation, Conv2D 
from tensorflow.keras.applications import ResNet101V2
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, MaxPooling2D, Dense, Dropout
#K.clear_session()

In [8]:
train = "D:/gp project/plant disess/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train/"
valid = "D:/gp project/plant disess/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid/"


In [9]:
%%time
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(featurewise_center=True)
datagen.mean = [123.68, 116.779, 103.939]
train_it = datagen.flow_from_directory(train,                           
    class_mode='categorical', batch_size=32, target_size=(224, 224))
test_it = datagen.flow_from_directory(valid,
        class_mode='categorical', batch_size=32, target_size=(224, 224))

Found 70295 images belonging to 38 classes.
Found 17572 images belonging to 38 classes.
CPU times: total: 1.7 s
Wall time: 5.32 s


In [23]:
target_names=train_it.class_indices
print(target_names);print()
print('len(target_names',len(target_names))

{'Apple___Apple_scab': 0, 'Apple___Black_rot': 1, 'Apple___Cedar_apple_rust': 2, 'Apple___healthy': 3, 'Blueberry___healthy': 4, 'Cherry_(including_sour)___Powdery_mildew': 5, 'Cherry_(including_sour)___healthy': 6, 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot': 7, 'Corn_(maize)___Common_rust_': 8, 'Corn_(maize)___Northern_Leaf_Blight': 9, 'Corn_(maize)___healthy': 10, 'Grape___Black_rot': 11, 'Grape___Esca_(Black_Measles)': 12, 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)': 13, 'Grape___healthy': 14, 'Orange___Haunglongbing_(Citrus_greening)': 15, 'Peach___Bacterial_spot': 16, 'Peach___healthy': 17, 'Pepper,_bell___Bacterial_spot': 18, 'Pepper,_bell___healthy': 19, 'Potato___Early_blight': 20, 'Potato___Late_blight': 21, 'Potato___healthy': 22, 'Raspberry___healthy': 23, 'Soybean___healthy': 24, 'Squash___Powdery_mildew': 25, 'Strawberry___Leaf_scorch': 26, 'Strawberry___healthy': 27, 'Tomato___Bacterial_spot': 28, 'Tomato___Early_blight': 29, 'Tomato___Late_blight': 30, 'Tomato

# VGG16 model below - essentially what Jason Brownlee used in his dogs versus cats piece

In [5]:
# define cnn model
from keras.applications.vgg16 import VGG16
from keras.models import Model
def define_model():
    # load model
    model = VGG16(include_top=False, input_shape=(224, 224, 3))
    # mark loaded layers as not trainable
    for layer in model.layers:
        layer.trainable = False
    # add new classifier layers
    flat1 = Flatten()(model.layers[-1].output)
    class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
    output = Dense(38, activation='softmax')(class1)
    # define new model
    model = Model(inputs=model.inputs, outputs=output)
    return model

In [6]:
model = define_model()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [7]:
%%time
epochsteps=len(train_it);epochs =10;valsteps=len(test_it)
# fit model
history = model.fit(train_it, steps_per_epoch=epochsteps,
                    validation_data=test_it, validation_steps=valsteps, epochs=10, verbose=1)
_, acc = model.evaluate(test_it, steps=75, verbose=1)
print('accuracy',acc * 100.0)


Epoch 1/10
2197/2197 [==============================] - 8145s 4s/step - loss: 1.0986 - accuracy: 0.7024 - val_loss: 0.5656 - val_accuracy: 0.8369
Epoch 2/10
2197/2197 [==============================] - 8101s 4s/step - loss: 0.4320 - accuracy: 0.8698 - val_loss: 0.5004 - val_accuracy: 0.8663
Epoch 3/10
2197/2197 [==============================] - 8185s 4s/step - loss: 0.3054 - accuracy: 0.9105 - val_loss: 0.4843 - val_accuracy: 0.8835
Epoch 4/10
2197/2197 [==============================] - 8193s 4s/step - loss: 0.2577 - accuracy: 0.9305 - val_loss: 0.5250 - val_accuracy: 0.8931
Epoch 5/10
2197/2197 [==============================] - 8188s 4s/step - loss: 0.2119 - accuracy: 0.9446 - val_loss: 0.5868 - val_accuracy: 0.8945
Epoch 6/10
2197/2197 [==============================] - 7953s 4s/step - loss: 0.1891 - accuracy: 0.9539 - val_loss: 0.6781 - val_accuracy: 0.8963
Epoch 7/10
2197/2197 [==============================] - 7998s 4s/step - loss: 0.1759 - accuracy: 0.9597 - val_loss: 0.6841 -

In [8]:
model.save('VGG16_model1.h5')
# not familiar how the above would work in terms of retrieving the model
# so I'll also try this
model1 = model 

In [11]:
test =  "D:/gp project/plant disess/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/test"



In [12]:
os.listdir(test)

['AppleCedarRust1.JPG',
 'AppleCedarRust2.JPG',
 'AppleCedarRust3.JPG',
 'AppleCedarRust4.JPG',
 'AppleScab1.JPG',
 'AppleScab2.JPG',
 'AppleScab3.JPG',
 'CornCommonRust1.JPG',
 'CornCommonRust2.JPG',
 'CornCommonRust3.JPG',
 'PotatoEarlyBlight1.JPG',
 'PotatoEarlyBlight2.JPG',
 'PotatoEarlyBlight3.JPG',
 'PotatoEarlyBlight4.JPG',
 'PotatoEarlyBlight5.JPG',
 'PotatoHealthy1.JPG',
 'PotatoHealthy2.JPG',
 'TomatoEarlyBlight1.JPG',
 'TomatoEarlyBlight2.JPG',
 'TomatoEarlyBlight3.JPG',
 'TomatoEarlyBlight4.JPG',
 'TomatoEarlyBlight5.JPG',
 'TomatoEarlyBlight6.JPG',
 'TomatoHealthy1.JPG',
 'TomatoHealthy2.JPG',
 'TomatoHealthy3.JPG',
 'TomatoHealthy4.JPG',
 'TomatoYellowCurlVirus1.JPG',
 'TomatoYellowCurlVirus2.JPG',
 'TomatoYellowCurlVirus3.JPG',
 'TomatoYellowCurlVirus4.JPG',
 'TomatoYellowCurlVirus5.JPG',
 'TomatoYellowCurlVirus6.JPG']

In [13]:
df_test = pd.DataFrame()

test_image_names = os.listdir(test)

len(test_image_names)

df_test['test_image_names'] = test_image_names

df_test

df_test['image_path'] = ''

df_test['predicted'] =''

In [14]:
df_test.head()

,test_image_names,image_path,predicted
0,AppleCedarRust1.JPG,,
1,AppleCedarRust2.JPG,,
2,AppleCedarRust3.JPG,,
3,AppleCedarRust4.JPG,,
4,AppleScab1.JPG,,


In [15]:
from pathlib import Path
for idx,row in df_test.iterrows():
    fname=Path(row[0])
    path = Path(test,fname)
    df_test.iloc[idx, 1] = path
    

In [16]:
df_test.head()

,test_image_names,image_path,predicted
0,AppleCedarRust1.JPG,D:\gp project\plant disess\New Plant Diseases ...,
1,AppleCedarRust2.JPG,D:\gp project\plant disess\New Plant Diseases ...,
2,AppleCedarRust3.JPG,D:\gp project\plant disess\New Plant Diseases ...,
3,AppleCedarRust4.JPG,D:\gp project\plant disess\New Plant Diseases ...,
4,AppleScab1.JPG,D:\gp project\plant disess\New Plant Diseases ...,


In [27]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
def load_image(filename):
    # load the image
    img = load_img(filename, target_size=(224, 224))
    # convert to array
    img = img_to_array(img)
    # reshape into a single sample with 3 channels
    img = img.reshape(1, 224, 224, 3)
    # center pixel data
    img = img.astype('float32')
    img = img - [123.68, 116.779, 103.939]
    return img

In [24]:
target_list = list(target_names.items());print(target_list)

[('Apple___Apple_scab', 0), ('Apple___Black_rot', 1), ('Apple___Cedar_apple_rust', 2), ('Apple___healthy', 3), ('Blueberry___healthy', 4), ('Cherry_(including_sour)___Powdery_mildew', 5), ('Cherry_(including_sour)___healthy', 6), ('Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 7), ('Corn_(maize)___Common_rust_', 8), ('Corn_(maize)___Northern_Leaf_Blight', 9), ('Corn_(maize)___healthy', 10), ('Grape___Black_rot', 11), ('Grape___Esca_(Black_Measles)', 12), ('Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 13), ('Grape___healthy', 14), ('Orange___Haunglongbing_(Citrus_greening)', 15), ('Peach___Bacterial_spot', 16), ('Peach___healthy', 17), ('Pepper,_bell___Bacterial_spot', 18), ('Pepper,_bell___healthy', 19), ('Potato___Early_blight', 20), ('Potato___Late_blight', 21), ('Potato___healthy', 22), ('Raspberry___healthy', 23), ('Soybean___healthy', 24), ('Squash___Powdery_mildew', 25), ('Strawberry___Leaf_scorch', 26), ('Strawberry___healthy', 27), ('Tomato___Bacterial_spot', 28), ('Toma

In [4]:
from keras.models import load_model
model = load_model('VGG16_model1.h5')
# that worked!

In [19]:
preds1 = []
# predict the class

for idx,row in df_test.iterrows():
    source=row[1]
    #print(source)
    img = load_image(source)
    result = model.predict(img)
    result = target_list[np.argmax(result)]
    preds1.append(result)   

1/1 [==============================] - 0s 112ms/step


In [20]:
df_test.predicted = preds1

In [21]:
df_test

,test_image_names,image_path,predicted
0,AppleCedarRust1.JPG,D:\gp project\plant disess\New Plant Diseases ...,"(Apple___Cedar_apple_rust, 2)"
1,AppleCedarRust2.JPG,D:\gp project\plant disess\New Plant Diseases ...,"(Apple___Cedar_apple_rust, 2)"
2,AppleCedarRust3.JPG,D:\gp project\plant disess\New Plant Diseases ...,"(Apple___Cedar_apple_rust, 2)"
3,AppleCedarRust4.JPG,D:\gp project\plant disess\New Plant Diseases ...,"(Apple___Cedar_apple_rust, 2)"
4,AppleScab1.JPG,D:\gp project\plant disess\New Plant Diseases ...,"(Apple___Apple_scab, 0)"
5,AppleScab2.JPG,D:\gp project\plant disess\New Plant Diseases ...,"(Apple___Apple_scab, 0)"
6,AppleScab3.JPG,D:\gp project\plant disess\New Plant Diseases ...,"(Squash___Powdery_mildew, 25)"
7,CornCommonRust1.JPG,D:\gp project\plant disess\New Plant Diseases ...,"(Corn_(maize)___Common_rust_, 8)"
8,CornCommonRust2.JPG,D:\gp project\plant disess\New Plant Diseases ...,"(Corn_(maize)___Common_rust_, 8)"
9,CornCommonRust3.JPG,D:\gp project\plant disess\New Plant Diseases ...,"(Corn_(maize)___Common_rust_, 8)"


In [22]:
# I see at least 3 wrong

In [23]:
train = "D:/gp project/plant disess/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train/"
valid = "D:/gp project/plant disess/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid/"

In [24]:
%%time
from os import makedirs
from os import listdir
from shutil import copyfile
# create directories
final_dir = 'final/'
#train = 'train'
#valid = 'valid'
directory1 = listdir(train);directory1.sort()
#print(directory1);print()
directory2 = listdir(valid);directory2.sort()
#print(directory2);print()
directory1 =directory1[1:];directory2=directory2[1:]
print(directory1 == directory2) 
final_subdirs =[]
for directory in directory1:
    final_sub = final_dir+directory
    final_subdirs.append(final_sub)
    #print('final_sub',final_sub)
    makedirs(final_sub, exist_ok=True)
print('len(final_subdirs)',len(final_subdirs))
i=0
for directory in directory1:
    train_sub = train+'/'+directory
    valid_sub = valid+'/'+directory
    #print(train_sub)
    train_files = listdir(train_sub)
    valid_files = listdir(valid_sub)
    for file in train_files:
        src = train_sub+'/'+file
        #print('src',src)
        dst = final_subdirs[i]+'/'+file
        #print('dst',dst)
        copyfile(src, dst)    
    for file in valid_files:
        src = valid_sub+'/'+file
        #print('src',src)
        dst = final_subdirs[i]+'/'+file
        #print('dst',dst)
        copyfile(src, dst)
    i+=1
        
        

True
len(final_subdirs) 37
CPU times: total: 23.2 s
Wall time: 6min 28s


In [1]:
def run_test_harness():
# define model
    model = define_model()

    datagen = ImageDataGenerator(featurewise_center=True)
    # specify imagenet mean values for centering
    datagen.mean = [123.68, 116.779, 103.939]
    # prepare iterator
    train_it = datagen.flow_from_directory('final/',
        class_mode='categorical', batch_size=32, target_size=(224, 224))
    # fit model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit_generator(train_it, steps_per_epoch=len(train_it), epochs=10, verbose=1)
    # save model
    model.save('VGG16_model2.h5')
    model2 = model

run_test_harness()

In [25]:
img = load_image('D:/gp project/plant disess/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/test/AppleCedarRust1.JPG')
result = model.predict(img)
result = target_list[np.argmax(result)]
print(result)

1/1 [==============================] - 0s 175ms/step
('Apple___Cedar_apple_rust', 2)


In [20]:
df_test['image_path'][0]

WindowsPath('D:/gp project/plant disess/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/test/AppleCedarRust1.JPG')